In [1]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset(
   'wikiann', 'en')

     |████████████████████████████████| 311 kB 7.0 MB/s 
     |████████████████████████████████| 243 kB 54.1 MB/s 
     |████████████████████████████████| 1.1 MB 54.6 MB/s 
     |████████████████████████████████| 133 kB 69.4 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 271 kB 64.2 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 74.3 MB/s 


Downloading:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/234M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wikiann downloaded and prepared to /root/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 20000
    })
})

In [3]:
import spacy
spacy.prefer_gpu()
!spacy download en_core_web_md 

import en_core_web_md 
model = en_core_web_md.load()


     |████████████████████████████████| 96.4 MB 1.2 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051301 sha256=7b085e9722d06d0f6a123f610f987c2c54f8edf94c8b6124a15580ddc67effd8
  Stored in directory: /tmp/pip-ephem-wheel-cache-rqm34sez/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [5]:
extraTags = [
             "PERSON",
             "GPE",
             "ORG",

]

spacyResult = []
datasetTkn = dataset['test']['tokens']
datasetTgs = dataset['test']['ner_tags']
 
for tokens,tags in list(zip(datasetTkn,datasetTgs)):
  text = ""
  for token,tag in list(zip(tokens,tags)):
    text +=token + " "
  doc = model(text)
  for token in doc:
      if token.is_space != True:
          if token.ent_type == 0:
                t = [(token.text,"O")]
                continue
          t = [(token.text,token.ent_type_)]
      spacyResult.append(t)



In [6]:
TagsIndex =[
  "O",
  "PERSON",
  "PERSON",
  "ORG",
  "ORG",
  "GPE",
  "GPE"
]

extraTags = [
             "PERSON",
             "GPE",
             "ORG",

]


actualData = []

datasetTkn = dataset['test']['tokens']
datasetTgs = dataset['test']['ner_tags']
 
for tokens,tags in list(zip(datasetTkn,datasetTgs)):
  text = ""
  for token,tag in list(zip(tokens,tags)):
    text += token + " "
    tag = TagsIndex[tag]
    if tag != "O":
      actualData.append([(token,tag)])

print(len(actualData),len(spacyResult))
      

39492 36039


In [7]:
actualData[100]

[('Hindalong', 'PERSON')]

In [8]:
spacyResult[100]

[('November', 'DATE')]

In [9]:
print(len(actualData),len(spacyResult))

39492 36039


In [10]:
import sklearn 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

tD = []
for a in spacyResult[:1000]:
  for b,c in a:
    tD.append(c)
aD = []
for d in actualData[:1000]:
  for e,f in d:
    aD.append(f)

results = confusion_matrix(aD, tD)
print ("Confusion Matrix :" )
print (results)
print("Accuracy Score : ")
accuracy_score(aD, tD)
print("Report: ")
print(classification_report(aD, tD))

Confusion Matrix :
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  8  36   1   6  33   0   5   1   2  46   4  78   0   3]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  4  38   4   1  41   3  10   2   1  79   3 140   1  10]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  9  58   1   3  40   0   5   0   3  46   1 267   0   7]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
Accuracy Score : 
Report: 
              precision    recall  f1-score   support

    CARDINAL       0.00      0.00      0.00         0
        DATE      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
